In [1]:
%%file chapter9_planning.lp

success :- goal(I),
I <= n.
:- not success.
    
1{occurs(A,I): action(A)}1 :- step(I),
    not goal (I),
    I < n.   

Writing chapter9_planning.lp


In [26]:
%%file chapter9_blocksplan.sp

#maxint=1000.
sorts
#block=[b][0..7].
#location=#block+{t}.
#inertial_fluent=on(#block(X),#location(Y)):X!=Y.
#defined_fluent=above(#block(X),#location(Y)):X!=Y.
#fluent=#inertial_fluent + #defined_fluent.
#action=put(#block(X),#location(Y)):X!=Y.
#step=0..8.
predicates
holds(#fluent,#step).
occurs(#action,#step).

rules
%initial:
holds(on(b0,t),0).
holds(on(b3,b0),0).
holds(on(b2,b3),0).
holds(on(b1,t),0).
holds(on(b4,b1),0).
holds(on(b5,t),0).
holds(on(b6,b5),0).
holds(on(b7,b6),0).
%% If block B is not known to be on location L at step 0,
%% then we assume it is not.
-holds(on(B,L),0) :- not holds(on(B,L),0).
holds(on(B,L),I+1) :- occurs(put(B,L),I).
%rule1
-holds(on(B,L2),I) :- holds(on(B,L1),I),L1 != L2.
%% rule 2
%Another useful rule says that no block can support more than one block
%directly on top of it
-holds(on(B2,B),I) :- holds(on(B1,B),I),B1 != B2,#block(B).
%inertia axioms:

holds(F,I+1) :- #inertial_fluent(F), holds(F,I), not -holds(F,I+1).
 
-holds(F,I+1) :- #inertial_fluent(F), -holds(F,I), not holds(F,I+1). 
    
%%rule3 - defining blocks above other blocks

holds(above(B,L),I) :- holds(on(B,L),I).
holds(above(B,L),I) :- holds(on(B,B1),I), holds(above(B1 ,L),I).
-holds(above(B,L),I) :-  #defined_fluent(F), not holds(above(B,L),I).

%impossibility:
-occurs(put(B,L),I) :- holds(on(B1,B),I). %% rule 5a
-occurs(put(B1,B),I) :- holds(on(B2,B),I), #block(B). %% rule 5b

%some actions:
occurs(put(b2,t),0).
occurs(put(b7,b2),1).
    
goal(I) :-
holds(on(b4 ,t),I), holds(on(b6 ,t),I),
holds(on(b1 ,t),I), holds(on(b3 ,b4),I),
holds(on(b7 ,b3),I), holds(on(b2 ,b6),I),
holds(on(b0 ,b1),I), holds(on(b5 ,b0),I).
    
success :- goal(I), I <= n.
:- not success.

Overwriting chapter9_blocksplan.sp


In [27]:
! java -jar sparc.jar chapter9_blocksplan.sp

SPARC  V2.57
chapter9_blocksplan.sp: predicate goal of arity 1 at line 55, column 1 was not declared


In [31]:
%%file chapter9_bwplans.lp

block(b0). block(b1). block(b2). block(b3).
block(b4). block(b5). block(b6). block(b7).

location(X) :- block(X).
location(t).

fluent( inertial , on(B,L)) :- block(B), location(L).
fluent( defined , above (B,L)) :- block(B), location(L).
action(put(B,L)) :- block(B), location(L),
B != L.

% initial assumptions

holds(on(b0 ,t) ,0).
holds(on(b3 ,b0) ,0).
holds(on(b2 ,b3) ,0).
holds(on(b1 ,t) ,0).
holds(on(b4 ,b1) ,0).
holds(on(b5 ,t) ,0).
holds(on(b6 ,b5) ,0).
holds(on(b7 ,b6) ,0).

%% If block B is not known to be on location L at
%% step 0, then we assume it is not.
-holds(on(B,L),0) :- block(B), location(L),
not holds(on(B,L) ,0).

#const n = 8.
step(0..n).

holds(on(B,L),I+1) :- occurs(put(B,L),I),
I < n.

-holds(on(B,L2),I) :- holds(on(B,L1),I),
location(L2),
L1 != L2.

-holds(on(B2 ,B),I) :- holds(on(B1 ,B),I),
block (B),
block(B2),
B1 != B2.

holds(above(B2 ,B1),I) :- holds(on(B2 ,B1),I).
    
holds(above(B2 ,B1),I) :- holds(on(B2 ,B),I),
holds(above(B,B1),I).

-occurs(put(B,L),I) :- location(L), holds(on(B1, B), I).

-occurs(put(B1, B), I) :- block(B1), block(B), holds(on(B2, B), I).

%% CWA for Defined Fluents
-holds(F, I) :- fluent(defined, F), step(I), not holds(F, I).

%% General Inertia Axiom
holds(F, I + 1) :- fluent(inertial, F), holds(F, I), not -holds(F, I + 1), I < n.
-holds(F, I + 1) :- fluent(inertial, F), -holds(F, I), not holds(F, I + 1), I < n.

%% CWA for Actions
-occurs(A, I) :- action(A), step(I), not occurs(A, I).

goal(I) :-
holds(on(b4 ,t),I), holds(on(b6 ,t),I),
holds(on(b1 ,t),I), holds(on(b3 ,b4),I),
holds(on(b7 ,b3),I), holds(on(b2 ,b6),I),
holds(on(b0 ,b1),I), holds(on(b5 ,b0),I).

#show occurs/2.

Overwriting chapter9_bwplans.lp


In [32]:
! clingo 0 chapter9_bwplans.lp --verbose

clingo version 5.7.1
Reading from chapter9_bwplans.lp
Reading      : file: chapter9_bwplans.lp
************** parsed program **************
block(b0).
block(b1).
block(b2).
block(b3).
block(b4).
block(b5).
block(b6).
block(b7).
location(t).
holds(on(b0,t),0).
holds(on(b3,b0),0).
holds(on(b2,b3),0).
holds(on(b1,t),0).
holds(on(b4,b1),0).
holds(on(b5,t),0).
holds(on(b6,b5),0).
holds(on(b7,b6),0).
location(X):-block(X).
fluent(inertial,on(B,L)):-block(B);location(L).
fluent(defined,above(B,L)):-block(B);location(L).
action(put(B,L)):-block(B);location(L);B!=L.
(-holds(on(B,L),0)):-block(B);location(L);not holds(on(B,L),0).
step((0..n)).
holds(on(B,L),(I+1)):-occurs(put(B,L),I);I<n.
(-holds(on(B,L2),I)):-holds(on(B,L1),I);location(L2);L1!=L2.
(-holds(on(B2,B),I)):-holds(on(B1,B),I);block(B);block(B2);B1!=B2.
holds(above(B2,B1),I):-holds(on(B2,B1),I).
holds(above(B2,B1),I):-holds(on(B2,B),I);holds(above(B,B1),I).
(-occurs(put(B,L),I)):-location(L);holds(on(B1,B),I).
(-occurs(put(B1,B),I)):-